# Atividade 3

## Análise e classificação de texto

Nesta atividade iremos utilizar um _dataset_ chamado __IMDB__ composto de opiniões sobre filmes em formato textual. O objetivo é classificar estes _inputs_ de forma binária (0 e 1) onde identificaremos se a opinião é positiva ou negativa.

__IMDB__ (Internet Movie DataBase) conta com um total de 50.000 opiniões sobre filmes onde separamos 25.000 delas para treino e os 50% restantes para teste. As frações são balanceadas, ou seja, contem um número igual de opiniões positivas e negativas.

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

## Importando a base para o projeto

Podemos obter os dados utilizando o facilitador `keras.datasets`, devemos lembrar que as palavras já foram trabalhadas e estão num formato numérico. 

Vamos apenas buscar as 10.000 palavras mais frequentes para facilitar no processamento.

In [ ]:
qtd_palavras = 10000

dataset = keras.datasets.imdb

(X_train, y_train), (X_test, y_test) = dataset.load_data(num_words = qtd_palavras)

## Visualizando os dados

Vamos entender o formato da informação que iremos utilizar para o trabalho. Esse é um passo principal e deve sempre ser realizado em qualquer processo analítico. Entender o formato do dado e qual é a importância de cada parcela da informação é primordial.

In [ ]:
qtd_base_treino = len(X_train)
qtd_meta_treino = len(y_train)
qtd_base_teste  = len(X_test)
qtd_meta_teste  = len(y_test)

txt = '''\
Quantidade de entradas e saídas para treino: {0} - {1}
Quantidade de entradas e saídas para teste:  {2} - {3}
'''.format(qtd_base_treino, qtd_meta_treino, qtd_base_teste, qtd_meta_teste)

print(txt)

Como foi dito acima, as palavras já foram transformadas em uma representação numérica, vamos imprimir o primeiro índice do array para ter uma ideia do formato do dado:

In [ ]:
print(X_train[0])

### Numerais para palavras

Podemos criar uma função que recebe de entrara um vetor e retorna o mesmo vetor trocando os numerais por seu representativo textual.

O método `get_word_index()` retorna o dicionário (chave : valor) que faz a tradução das palavras. Os textos tem algumas _tags_ que são utilizadas como auxiliares na etapa inicial do processamento do texto, são elas: `<PAD>`, `<START>`, `<UNK>` e `<UNUSED>`. As posições iniciais do dicionário de palavras são reservadas para as _tags_, devemos levar isso em consideração.

In [ ]:
dic_palavras = dataset.get_word_index()

dic_palavras = {chave: (valor + 3) for chave, valor in dic_palavras.items()} 

dic_palavras["<PAD>"]    = 0
dic_palavras["<START>"]  = 1
dic_palavras["<UNK>"]    = 2
dic_palavras["<UNUSED>"] = 3

dic_palavras_formatado = dict([(valor, chave) for (chave, valor) in dic_palavras.items()])

def decode_palavras(vetor_palavras):
    return ' '.join([dic_palavras_formatado.get(palavra, '?') for palavra in vetor_palavras])

Vamos utilizar nossa função auxiliar `decode_palavras()` para traduzir o primeiro índice de nosso conjunto de treino `X_train`, veja o resultado:

In [ ]:
decode_palavras(X_train[0])

Verifique o texto de saída:

```
"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all"
```

Note que as _tags_ auxiliam a formatação do conteúdo, palavras que não foram corretamente processadas na etapa inicial do processo ou vieram com algum tipo de "sujeira" foram trocadas pela _tag_ `<UNK>` de _unknow_ (desconhecido).

Tradicionalmente, o processamento de texto inicial envolve um processo de remoção das _stopwords_, conversão das palavras para caixa baixa, remoção de pontuação e no caso do português, remoção da acentuação gráfica de palavras.

## Preparando os vetores para a batalha

Sabemos que uma rede neural espera um tensor de entrada, e vamos verificar uma coisa antes de continuar:

In [ ]:
print('{} - {} - {}'.format(len(X_train[0]), len(X_train[1]), len(X_train[2])))

Verifique que imprimimos o tamanho dos vetores em `X_train` nas posições 0, 1 e 2. Os valores que obtivemos foram 218, 189 e 141 respectivamente.

Temos um problema pois uma rede neural espera um tensor de tamanho único na entrada. Podemos resolver este problema de algumas formas, por exemplo:

* __OneHot-Encode__: Os vetores serão convertidos em vetores binários (0 e 1) de dimensão igual ao número total de palavras que temos em nosso dicionário de palavras. Há benefícios em relação a velocidade na busca dos valores transformados, porém, existe um alto uso de memória já que teremos uma matriz de dimensão $\text{número de palavras} \times \text{número de opiniões}$.
* __PadSequence__: Os vetores são preenchidos com de forma que tenham o tamanho do maior vetor da nossa coleção de documentos, ou seja, teremos um tensor de dimensão $\text{tamanho do maior vetor} \times \text{número de opiniões}$. Podemos criar uma camada de _embedding_ como camada inicial de nossa rede para processar essa informação de entrada, algo similar ao que vimos no exemplo do _Word2Vec_.

A biblioteca _tensorflow_ tem um facilitador para aplicar [pad_sequences](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) aos vetores, vamos utilizar essa abordagem para padronizar o tamanho dos vetores de _input_.

In [ ]:
len(X_train.max())

In [ ]:
len(X_test.max())

In [ ]:
input_dim = 894


X_train = keras.preprocessing.sequence.pad_sequences(X_train,
                                                     value = dic_palavras["<PAD>"],
                                                     padding = 'post',
                                                     maxlen = input_dim)

X_test = keras.preprocessing.sequence.pad_sequences(X_test,
                                                    value = dic_palavras["<PAD>"],
                                                    padding = 'post',
                                                    maxlen = input_dim)

Vamos verificar as dimensões após a operação:

In [ ]:
len(X_train[0])

In [ ]:
print(X_train[0])

## Construindo o modelo

Lembre que nossa primeira camada será uma camada de `embedding` que deve ser capaz de traduzir todas as palavras de nosso dicionário de palavras, por isso, esta camada deve ter uma dimensão de entrada igual ao número total de palavras no nosso dicionário.

Utilizaremos uma camada convolucional de uma dimensão para criar um vetor representativo da nossa informação, lembre que teremos uma representação vetorial para cada palavra agora.

A camada de saída dessa rede deve resultar em um valor binário (0 e 1), por este motivo, vamos utilizar uma função `sigmoid` na saída da rede.

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Embedding(qtd_palavras, 16, input_length = input_dim))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation = tf.nn.relu))
model.add(keras.layers.Dense(1, activation = tf.nn.sigmoid))

model.summary()

### Função loss

Como estamos utilizando uma função `sigmoid` na camada de saída, vamos utilizar `binary_crossentropy` como função de loss. Essa função calcula qual é a distância entre o que o modelo deveria ter dado como resposta e a sua atual resposta.

In [ ]:
model.compile(optimizer = tf.train.AdamOptimizer(),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

## Treino

Vamos treinar o modelo por 60 épocas em batches de 512 amostras. Isso quer dizer que em 60 iterações o modelo vai receber todos os tensores de `X_train` e vai corrigir a saída comparando o resultado com o que existe em `y_train`, 512 exemplos por vez. Enquanto estas 60 iterações ocorrem, vamos testar o modelo com 20% dos tensores de `X_train`, essa é a nossa parcela de validação.

In [ ]:
history = model.fit(X_train,
                    y_train,
                    epochs = 60,
                    batch_size = 512,
                    validation_split = 0.2,
                    verbose = 1)

## Hora da verdade

Através do método `evaluate(input, output)` é possível validar como o modelo está respondendo a um determinado conjunto de dados. No nosso caso, vamos verificar como o modelo se comporta com a parcela de informação que temos em `X_test`.

O método retorna um vetor, na primeira posição temos o valor calculado para o loss do modelo e na segunda posição a acurácia.

In [ ]:
resultados = model.evaluate(X_test, y_test)

print(resultados)
print('-------- * --------')
print('Loss teste: {:.4f}'.format(resultados[0]))
print(' Acc teste: {:.2f}%'.format(resultados[1]*100))

## Plot dos resultados de saída

In [ ]:
history_dict = history.history

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(1, figsize = (15, 5))

plt.subplot(121)
plt.plot(epochs, loss, 'r', label = 'Loss treino')
plt.plot(epochs, val_loss, 'b', label = 'Loss validação')
plt.title('Loss: Treino e validação')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']
plt.subplot(122)
plt.plot(epochs, acc, 'r', label = 'Acc treino')
plt.plot(epochs, val_acc, 'b', label = 'acc validação')
plt.title('Acc: Treino e validação')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

# RNN

Vamos resolver o mesmo problema acima, desta vez com uma RNN.

In [ ]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout
from tensorflow.keras.layers import LSTM, CuDNNLSTM

Vamos reduzir a quantidade de palavras de entrara por uma questão de recursos para processamento na plataforma do Colab.

In [ ]:
(X_train, y_train), (X_test, y_test) = dataset.load_data(num_words = qtd_palavras)

X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen = 80)

X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen = 80)

O modelo é construído da mesma forma, uma novidade é a camada `LSTM`.

Essa classe da biblioteca `Keras` implementa uma célula de memória e podemos "empilhar" essas células. Sempre lembrando que todas, exceto a ultima da cadeia, deve ter o parâmetro `return_sequences = True`.

In [ ]:
model_RNN = Sequential()
model_RNN.add(Embedding(qtd_palavras, 50, input_length = 80))
# model_RNN.add(LSTM(32))
model_RNN.add(CuDNNLSTM(32)) # Caso GPU
model.add(Dropout(0.2))

#model_RNN.add(LSTM(32, return_sequences=True))
#model_RNN.add(LSTM(32, return_sequences=True))
#model_RNN.add(LSTM(32))
model_RNN.add(Dense(1, activation = tf.nn.sigmoid))

model_RNN.summary()

In [ ]:
model_RNN.compile(optimizer = tf.train.AdamOptimizer(),
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy'])

In [ ]:
history_RNN = model_RNN.fit(X_train,
                            y_train,
                            epochs = 5,
                            batch_size = 128,
                            validation_split = 0.2,
                            verbose = 1)

In [ ]:
resultados_RNN = model_RNN.evaluate(X_test, y_test)

print(resultados_RNN)
print('-------- * --------')
print('Loss teste: {:.4f}'.format(resultados_RNN[0]))
print(' Acc teste: {:.2f}%'.format(resultados_RNN[1]*100))

In [ ]:
history_dict_RNN = history_RNN.history

acc = history_RNN.history['acc']
val_acc = history_RNN.history['val_acc']
loss = history_RNN.history['loss']
val_loss = history_RNN.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(1, figsize = (15, 5))

plt.subplot(121)
plt.plot(epochs, loss, 'r', label = 'Loss treino')
plt.plot(epochs, val_loss, 'b', label = 'Loss validação')
plt.title('Loss: Treino e validação')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']
plt.subplot(122)
plt.plot(epochs, acc, 'r', label = 'Acc treino')
plt.plot(epochs, val_acc, 'b', label = 'acc validação')
plt.title('Acc: Treino e validação')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()